In [ ]:
# load
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import KFold
import re
tqdm.pandas(desc="my bar!")

pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
def wlogloss(targets, preds):
    target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']
    res = 0
    for col in target_cols:
        res += log_loss(targets[col], preds[col+'_pred'])
        if col == 'any':
            res += log_loss(targets[col], preds[col+'_pred'])
    res /= 7
    return res
target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']


In [ ]:
with open('stack_feats_ts.pickle', 'rb') as f:
    test_list = pickle.load(f)
with open('stack_feats_tr.pickle', 'rb') as f:
    train = pickle.load(f)

In [ ]:
with open('train_6model_fasterfeats.pickle', 'rb') as f:
    train_ = pickle.load(f)


In [ ]:
train = pd.merge(train, train2.drop(['folds', 'StudyInstanceUID'] + target_cols, axis=1), on='ID', how='left')

In [ ]:
X_cols = train.columns.drop(['ID', 'folds', 'StudyInstanceUID'] + target_cols)
# X_cols = test_list[0].columns.drop(['ID'])
len(X_cols)

In [ ]:
for c in target_cols:
    del train[c]
    train[c] = train_[c]

In [ ]:
%%time
from sklearn.linear_model import Ridge, LogisticRegression
stack_preds = []
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": 0.1,
          "num_leaves": 5,
           "max_bin": 256,
          "feature_fraction": 0.8,
          "verbosity": 0,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }
res = []
pred_test = []
fi = {}
for c in target_cols:
    fi[c] = []
stack_preds_valid = []
for i in range(5):
    tr = train.query('folds != @i')
    va = train.query('folds == @i')
    preds = pd.DataFrame(np.zeros([len(va), 6]), columns = [c + '_pred' for c in target_cols])
    for tar_col in target_cols:
        tr_D = lgb.Dataset(tr[X_cols], tr[tar_col])
        va_D = lgb.Dataset(va[X_cols], va[tar_col])
        clf = lgb.train(params, tr_D, 10000, valid_sets=va_D, verbose_eval=100,
                                    early_stopping_rounds=120)
        preds[tar_col + '_pred'] = clf.predict(va[X_cols])
#         pred_test.append(clf.predict(test_list[i][X_cols]))
        df_fi = pd.DataFrame(clf.feature_importance(importance_type='gain'), index = X_cols, columns=['FI_score'])
        fi[tar_col].append(df_fi)
        print(log_loss(va[tar_col], preds[tar_col + '_pred']))
        res.append(log_loss(va[tar_col], preds[tar_col + '_pred']))
        print('-'*80)
    print('='*80)
    stack_preds.append([va['ID'], preds])

In [ ]:
def get_score(res):
    score = 0
    for i, r in enumerate(res):
        if i %6 == 0:
            score += 2*r
        else:
            score += r
    return score/5/7
get_score(res)